# Toy experiments

This notebook displays regions obtained by different multi-output conformal methods

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import matplotlib.pyplot as plt
import torch
from tqdm import tqdm

from moc.models.train import train
from moc.configs.config import get_config
from moc.utils.general import seed_everything, savefig
from moc.utils.run_config import RunConfig
from moc.analysis.dataframes import load_datamodule
from moc.analysis.plot_2d_vs_1d import plot_2D_region_vs_1D_per_method, plot_2D_region_vs_1D_per_model, plot_2D_region_vs_1D_for_C_HDR_per_n_samples
from moc.analysis.plot_2d import plot_2D_region_per_method, plot_2D_regions_by_x_and_tau
from moc.datamodules.toy_datamodule import UnimodalHeteroscedastic, BimodalHeteroscedastic
from moc.analysis.utils import get_oracle_model

name = 'toy'
path = Path('results') / name
path.mkdir(exist_ok=True, parents=True)
seed_everything(0)

In [2]:
config = get_config()
config.device = 'cuda' # 'cuda' or 'cpu'

## Unimodal dataset

### Model training

In [ ]:
dataset = 'unimodal_heteroscedastic'
model_name = 'MQF2'
rc = RunConfig(config, 'toy_2dim', dataset, 0, hparams={'model': model_name})
datamodule = load_datamodule(rc)
oracle_model = get_oracle_model(rc, datamodule)
mqf2_model = train(rc, datamodule)

### 3D plots of 2D output regions vs 1D input

In [ ]:
# List of methods with default hyperparameters
methods = ['M-CP', 'CopulaCPTS', 'DR-CP', 'C-HDR', 'PCP', 'HD-PCP', 'STDQR', 'C-PCP', 'L-CP']
hparams_list = [{'posthoc_method': method} for method in methods]

torch.manual_seed(0)
plot_path = path / 'contours' / f'{model_name}_{dataset}.pdf'
plot_2D_region_vs_1D_per_method(hparams_list, datamodule, config, oracle_model, mqf2_model, path=plot_path, grid_side=300)

### 2D plots of 2D output regions

In [ ]:
plot_2D_regions_by_x_and_tau(datamodule, oracle_model, mqf2_model, path=path / 'contours' / f'{dataset}_conf_slices.pdf', grid_side=100)

## Bimodal dataset

### Model training

In [ ]:
dataset = 'bimodal_heteroscedastic'
model_name = 'MQF2'
rc = RunConfig(config, 'toy_2dim', dataset, 0, hparams={'model': model_name})
datamodule = load_datamodule(rc)
oracle_model = get_oracle_model(rc, datamodule)
mqf2_model = train(rc, datamodule)

### 3D plots of 2D output regions vs 1D input

In [ ]:
methods = ['M-CP', 'CopulaCPTS', 'DR-CP', 'C-HDR', 'PCP', 'HD-PCP', 'STDQR', 'C-PCP', 'L-CP']
hparams_list = [{'posthoc_method': method} for method in methods]

torch.manual_seed(0)
plot_path = path / 'contours' / f'{model_name}_{dataset}.pdf'
plot_2D_region_vs_1D_per_method(hparams_list, datamodule, config, oracle_model, mqf2_model, path=plot_path, grid_side=300)

### 2D plots of 2D output regions

In [ ]:
plot_2D_regions_by_x_and_tau(datamodule, oracle_model, mqf2_model, path=path / 'contours' / f'{dataset}_conf_slices.pdf', grid_side=100)

## Models comparison on a bimodal dataset

### Models training

In [ ]:
from moc.models.train import models, trainers

dataset = 'bimodal_heteroscedastic'
datamodule = load_datamodule(rc)

models_order = ['Mixture', 'DRF-KDE', 'MQF2']
trained_models = {}
for model_name in models_order:
    rc = RunConfig(config, 'toy_2dim', dataset, 0, hparams={'model': model_name})
    trained_models[model_name] = train(rc, datamodule)
    print(f'Finished training {rc.summary_str()}')

### 3D plots of 2D output regions vs 1D input

In [ ]:
oracle_model = get_oracle_model(rc, datamodule)

torch.manual_seed(0)
plot_2D_region_vs_1D_per_model(datamodule, config, oracle_model, trained_models, path=path / 'contours' / f'{dataset}_per_model.pdf', grid_side=300)

## Moon dataset

In [ ]:
dataset = 'one_moon_heteroscedastic'
model_name = 'MQF2'
rc = RunConfig(config, 'toy_2dim', dataset, 0, hparams={'model': model_name})
datamodule = load_datamodule(rc)
oracle_model = get_oracle_model(rc, datamodule)
mqf2_model = train(rc, datamodule)

### 3D plots of 2D output regions vs 1D input

In [ ]:
methods = ['M-CP', 'CopulaCPTS', 'DR-CP', 'C-HDR', 'PCP', 'HD-PCP', 'STDQR', 'C-PCP', 'L-CP']
hparams_list = [{'posthoc_method': method} for method in methods]

torch.manual_seed(0)
plot_path = path / 'contours' / f'{model_name}_{dataset}.pdf'
plot_2D_region_vs_1D_per_method(hparams_list, datamodule, config, oracle_model, mqf2_model, path=plot_path, grid_side=300)

### 2D plots of 2D output regions

In [ ]:
torch.manual_seed(0)
fig, ax = plt.subplots(1, 1, figsize=(3, 3))
plot_2D_region_per_method(ax, -1.2, 0.8, datamodule, None, mqf2_model, grid_side=800, custom_xlim=(-3, 3), custom_ylim=(-3.2, 3.4))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1, prop={'family': plt.rcParams['font.family']}, frameon=False)
savefig(path / 'contours' / f'{dataset}_2d_x_1.pdf')

In [ ]:
plot_path = path / 'contours' / f'{model_name}_{dataset}_conf_slices.pdf'
plot_2D_regions_by_x_and_tau(datamodule, oracle_model, mqf2_model, path=plot_path, grid_side=100)

## Varying number of samples

### 3D plots of 2D output regions vs 1D input for C-HDR with varying number of samples

In [ ]:
torch.manual_seed(0)
plot_2D_region_vs_1D_for_C_HDR_per_n_samples(datamodule, config, oracle_model, path=path / 'contours' / f'{dataset}_C_HDR_per_n_samples.pdf', grid_side=300)